<a href="https://colab.research.google.com/github/kodebot/classical_tamil_llm/blob/main/Fine_tune_Tamil_LLaMA_7b_instruct_for_Tamil_Poem_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Tamil Llama 7b instruct for Tamil Poem


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "abhinand/tamil-llama-7b-instruct-v0.2"

# The instruction dataset to use
dataset_names = [
    "kodebot/Thirukural_tamil_with_meaning",
    "kodebot/Thirukural_tamil_with_meaning",
    "kodebot/Thirukural_tamil_with_meaning"
]

# Fine-tuned model name
new_model = "tamil-llama-7b-instruct-tamil-poem-experiment-2.1"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
dataset_split = dataset.train_test_split(test_size=0.1)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_split["train"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1197 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,5.291700
50,4.371000
75,4.076800
100,4.053700
125,3.941000
150,4.122000
175,3.870600
200,3.883700
225,3.875900
250,3.921500


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

generated_texts = []
reference_texts = []
for inst in dataset_split["test"]:
  # Run text generation pipeline with our next model
  splits = inst["text"].replace("<s>", "").replace("</s>", "").replace("[INST]", "").split("[/INST]")
  prompt = splits[0].strip()
  reference_texts.append(splits[1].strip())
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
  result = pipe(f"<s>[INST] {prompt} [/INST]")
  generated_texts.append(result[0]['generated_text'].replace("<s>", "").replace("</s>", "").replace("[INST]", "").split("[/INST]")[1])
  print(len(generated_texts), " of ", 10)
  if len(generated_texts) >= 10:
    break

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


1  of  10
2  of  10
3  of  10
4  of  10
5  of  10
6  of  10
7  of  10
8  of  10
9  of  10
10  of  10


In [ ]:
generated_texts

[' நல்லொழுக்கம் இல்லாதவர்க்கு, நல்லொழுக்கம் உடையவர்க்கு, தான்பிறந்தவர்க்கு, இல்லாமை உடையவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உலகில் உள்ளவர்க்கு, உல',
 ' உண்ணாமை வேண்டும், ஏனென்றால், உண்ணாமை இல்லாதவர், உண்ணாமை உடையவர்க்கு உண்ணாமை இல்லாதவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை இல்லாதவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை இல்லாதவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை இல்லாதவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை இல்லாதவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்கு உண்ணாமை உடையவர்க்

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d5dbe839bd8cd4704e1532630239c829f8ba53294dbfe3301e251c642b5f482b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=generated_texts,
                         references=reference_texts)
print(results)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [ ]:
# Empty VRAM
del model
del pipe
del trainer


In [ ]:

import gc
gc.collect()
gc.collect()

20933

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kodebot/tamil-llama-7b-instruct-tamil-poem-experiment-2.1/commit/e9290670b3acc9e2a9ad74c1bdbd52e886e98e9f', commit_message='Upload tokenizer', commit_description='', oid='e9290670b3acc9e2a9ad74c1bdbd52e886e98e9f', pr_url=None, pr_revision=None, pr_num=None)